# Dane testowe

Zacznę od paru importów:

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re

W internecie znalazłem najczęściej występujące nazwiska:

In [2]:
nazwiska = '''Nowak 92867
Kowalski 63635
Wiśniewski 50346
Wójcik 46052
Kowalczyk 45503
Kamiński 43473
Lewandowski 42170
Zieliński 41504
Woźniak 41101
Szymański 40648
Dąbrowski 39352
Kozłowski 34520
Jankowski 31405
Mazur 31271
Wojciechowski 30188
Kwiatkowski 30120
Krawczyk 29109
Kaczmarek 28620
Piotrowski 28035
Grabowski 26659
Zając 25955
Król 25395
Pawłowski 25315
Michalski 24790
Wróbel 23527
Jabłoński 22983
Wieczorek 22724
Nowakowski 22399
Majewski 22344
Olszewski 22078
Stępień 22040
Jaworski 21502
Dudek 21452
Adamczyk 21400
Malinowski 21398
Pawlak 20912
Górski 20727
Sikora 20533
Nowicki 20465
Witkowski 20202
Walczak 20193
Baran 19973
Rutkowski 19943
Michalak 19539
Szewczyk 19359
Ostrowski 19047
Tomaszewski 18346
Zalewski 18121
Wróblewski 17801
Pietrzak 17784
Jasiński
Zawadzki
Bąk
Jakubowski
Sadowski
Duda
Włodarczyk
Wilk
Chmielewski
Borkowski
Sokołowski
Szczepański
Sawicki
Kucharski
Lis
Maciejewski
Kubiak
Kalinowski
Mazurek
Wysocki
Kołodziej
Kaźmierczak
Czarnecki
Sobczak
Konieczny
Urbański
Głowacki
Wasilewski
Sikorski
Zakrzewski
Krajewski
Krupa
Laskowski
Ziółkowski
Gajewski
Mróz
Brzeziński
Szulc
Szymczak
Makowski
Baranowski
Przybylski
Kaczmarczyk
Borowski
Błaszyk
Adamski
Górecki
Chojnacki
Kania'''

Wyciągam listę nazwisk z powyższego tekstu:

In [3]:
nazwiska = [i.capitalize() for i in re.findall(r'(?i)([A-ZĄĘŁŃÓŚŹ]+)', nazwiska)]

Dorzucam najpopularniejsze imiona:

In [4]:
imiona = '''Jan
Stanisław
Andrzej
Józef
Tadeusz
Jerzy
Zbigniew
Krzysztof
Henryk
Ryszard
Kazimierz
Marek
Marian
Piotr
Janusz
Władysław
Adam
Wiesław
Zdzisław
Edward
Mieczysław
Roman
Mirosław
Grzegorz
Czesław
Dariusz
Wojciech
Jacek
Eugeniusz
Tomasz'''
imiona = imiona.split()

Z olxa wyciągam stanowiska:

In [5]:
stanowiska = '''Murarz
Tynkarz
Cieśla
Operator
Hydraulik
Elektryk
Spawacz
Technik
Glazurnik
Montażysta
Tynkarz'''
stanowiska = stanowiska.split()

I przykładowe zakresy płac dla poszczególnych stanowisk:

In [6]:
stawki = {'Murarz': (30, 35),
 'Elektryk': (22, 35),
 'Cieśla': (25, 30),
 'Technik': (30, 50),
 'Spawacz': (30, 60),
 'Tynkarz': (18, 25),
 'Operator': (25, 40),
 'Glazurnik': (18, 25),
 'Hydraulik': (25, 35),
 'Montażysta': (15, 25)}

Brakuje jeszcze listy z budowami:

In [7]:
budowy = '''Metro C1
Metro C2
Metro B20
Metro B21
Dworzec Wschodni
Dworzec Gdański
Obwodnica Południowa A
Obwodnica Południowa B
Obwodnica Południowa C
Varso
PKiN
Marynarska'''

budowy = budowy.split('\n')

I tworzę z nich losowych pracowników:

In [8]:
ilosc_pracownikow = 200

df = pd.DataFrame(0, index=range(ilosc_pracownikow), columns=['Imię', 'Nazwisko'])

df["Imię"] = np.random.choice(imiona, size=len(df))
df["Nazwisko"] = np.random.choice(nazwiska, size=len(df))

df['Pracownik'] = df["Nazwisko"] + ' ' + df["Imię"]
df = df.drop(['Imię', 'Nazwisko'], axis=1)

df["Stanowisko"] = np.random.choice(stanowiska, size=len(df))
df["Budowa"] = np.random.choice(budowy, size=len(df))

for i, row in df.iterrows():
    zakres = stawki[row['Stanowisko']]
    df.at[i,'Stawka']= np.random.randint(zakres[0], zakres[1] + 1, size=1)

df.head(10)

,Pracownik,Stanowisko,Budowa,Stawka
0,Duda Tadeusz,Cieśla,Dworzec Gdański,28.0
1,Kołodziej Jan,Operator,Obwodnica Południowa C,36.0
2,Król Piotr,Cieśla,PKiN,29.0
3,Wysocki Zdzisław,Cieśla,Metro B20,30.0
4,Kozłowski Krzysztof,Technik,Varso,50.0
5,Grabowski Ryszard,Tynkarz,Metro B20,19.0
6,Zakrzewski Tomasz,Spawacz,Obwodnica Południowa C,33.0
7,Sokołowski Piotr,Operator,Dworzec Gdański,32.0
8,Kania Zbigniew,Elektryk,Metro B20,26.0
9,Jaworski Grzegorz,Technik,Obwodnica Południowa C,37.0


In [9]:
df.to_csv('reports_employee.csv', header=False, encoding='utf-8')

Tworzę listę rekordów na *n* dni:

In [10]:
n = 62

dane = df.copy()
dane['Data'] = datetime.now().date()

for i in range(1, n):
    dzien = datetime.now().date() - timedelta(i)
    if dzien.weekday() not in {5, 6}:
        df['Data'] = dzien
        dane = pd.concat([dane,df], ignore_index=True)
godziny = np.random.randint(7, 10, size=len(dane))
dane['Dniówka'] = godziny * dane['Stawka']
dane = dane.drop(['Stawka'], axis=1)
dane.index += 1

Zapisuję dane do csv:

In [11]:
dane.to_csv('reports_record.csv', header=False, encoding='utf-8')